# Essentially same as otbn_find_bits.ipynb but streamlined for 25M captures.

In [ ]:
import numpy as np
#waves = np.load('waves_p256_streamed.npy')
#waves = np.load('waves_p256_streamed_half1half0.npy')
#waves = np.load('waves_p256_streamed_patterned.npy')
waves = np.load('waves_p256_streamed_32bitblocks.npy')
#wave = np.load('waves_p256_100M.npy')
#wave = np.load('waves_p256_100M_2s.npy')

In [ ]:
import numpy as np
import pandas as pd
from scipy import signal

def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq 
    b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def butter_highpass_filter(data, cutoff, fs, order=9):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = signal.filtfilt(b, a, data)
    return y

In [ ]:
filtered_waves = []
for w in waves:
    filtered_waves.append(butter_highpass_filter(w, 1e6, 100e6))

In [ ]:
def contiguous_regions(condition):
    """Finds contiguous True regions of the boolean array "condition". Returns
    a 2D array where the first column is the start index of the region and the
    second column is the end index."""

    # Find the indicies of changes in "condition"
    d = np.diff(condition.astype(int))
    idx, = d.nonzero() 

    # We need to start things after the change in "condition". Therefore, 
    # we'll shift the index by 1 to the right.
    idx += 1

    if condition[0]:
        # If the start of condition is True prepend a 0
        idx = np.r_[0, idx]

    if condition[-1]:
        # If the end of condition is True, append the length of the array
        idx = np.r_[idx, condition.size] # Edit

    # Reshape the result into two columns
    idx.shape = (-1,2)
    return idx


### Find runs of samples below threshold value, then guess at bit start times:
(keep only runs that are long enough)

In [ ]:
THRESHOLD = 0.1
MIN_RUN_LENGTH = 500
STOP=len(filtered_waves[0])

bit_startss = []

for w in filtered_waves:
    condition = np.abs(w[:STOP]) < THRESHOLD
    results = contiguous_regions(condition)
    goods = results[np.where(results[:,1] - results[:,0] > MIN_RUN_LENGTH)]

    bit_starts = []
    idle_count = 0
    max_active = 10
    for i, idle_period in enumerate(goods[1:]):
        #print('Processing start=%10d... ' % goods[i][0], end='')
        if goods[i][0] - goods[i-1][1] <= max_active:
            idle_count += 1
            #print('idle_count=%d for start=%d, stop=%d' % (idle_count, goods[i][0], goods[i][1]))
            if idle_count == 4:
                #print('Found bit starting at %d' % goods[i+1][0])
                bit_starts.append(goods[i][1]+1)
                idle_count = 0
        else:
            #print('resetting idle_count for start=%d, stop=%d because delta=%d (%d, %d)' % (goods[i][0], goods[i][1], goods[i][0] - goods[i-1][1], goods[i][0], goods[i-1][1]))
            idle_count = 1
    
    bit_startss.append(bit_starts)

In [ ]:
len(bit_startss[0])

### Sanity check the results:

In [ ]:
for bit_starts in bit_startss:
    if len(bit_starts) != 256:
        print('Oops, found %d bits :-/' % len(bit_starts))

    duration = bit_starts[1] - bit_starts[0]
    if duration != 27350:
        print("Unexpected first bit duration: %d" % duration)
    for b in range(2, len(bit_starts)):
        d = bit_starts[b] - bit_starts[b-1] 
        if d != duration:
            print("Unexpected duration for bit %d: %d cycles" % (b-1, d))


## Superimpose all bits!

In [ ]:
filtered_wave = filtered_waves[0]

bits = []
bit_size = bit_starts[1] - bit_starts[0]
for start in bit_starts:
    bits.append(filtered_wave[start:start+bit_size])

In [ ]:
import holoviews as hv
from holoviews.operation import decimate
from holoviews.operation.datashader import datashade, shade, dynspread
from holoviews import opts
import datashader as ds

hv.extension('bokeh')

lines = {i: hv.Curve(np.asarray(bits[i])) for i in range(len(bits[:22]))}
linespread = dynspread(datashade(hv.NdOverlay(lines, kdims='k'), aggregator=ds.by('k', ds.count())))
linespread.opts(opts.RGB(width=2400, height=1200))


In [ ]:
#numbits = len(bits) #slow but will work
numbits = 20

import holoviews as hv
from holoviews.operation import decimate
from holoviews.operation.datashader import datashade, shade, dynspread
hv.extension('bokeh')

xrange = range(duration)

from operator import mul
from functools import reduce

curves = [hv.Curve(zip(xrange, filtered_wave[bit_starts[i]:bit_starts[i]+duration])) for i in range(numbits)]

#reduce(mul, curves).opts(width=2000, height=900)
datashade(reduce(mul, curves)).opts(width=2000, height=900)

# Attack using markers from average 'one' and 'zero'

In [ ]:
# markers generated by otbn_find_bits.ipynb:
markers = np.load('markers_25M.npy')

In [ ]:
scores = []
for b in bit_starts:
    score = 0
    for marker in markers:
        score += filtered_waves[0][b + marker]
    scores.append(score)
    

In [ ]:
allscores = []
for b in range(256):
    score = 0
    for marker in markers:
        for i,w in enumerate(filtered_waves):
            score += w[bit_startss[i][b] + marker]
    allscores.append(score/len(filtered_waves))

In [ ]:
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

cscores = hv.Curve(scores)
callscores = hv.Curve(allscores)
#(cscores*callscores).opts(width=2000, height=600)
(cscores).opts(width=2000, height=600)